In [46]:
from bokeh.plotting import figure, output_file, show
import pandas as pd
import os
import numpy as np
from bokeh.io import output_notebook
from IPython.display import display, HTML
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.models.tools import CustomJSHover
from bokeh.palettes import Dark2_5 as palette
import itertools #for cycling through colors 
from bokeh.models import Legend
import sys

"""

"""

def generate_chart(csvs, metric):
    valid_metrics = {"p50": "p50", "p90": "p90", "p99": "p99"}#, "mem": "mem_MB_max_fortioserver_deployment_proxy", "cpu": "cpu_mili_avg_fortioserver_deployment_proxy"}

    if metric is None:
        sys.exit('need metric')
    if metric not in valid_metrics: 
        sys.exit("invalid metric")
    if csvs is None:
        sys.exit('need one or more CSV files') 
    
    m = valid_metrics[metric] #the CSV field to plot 
    files=csvs.split(",") 

    # 1. read in all rows in both files  
    df = read_csv(files) 

    # 2. generate series to plot (x=connections, y=metric) 
    c, series = get_series(df, m) 

    # 3. generate title 
    qps=df.at[0, 'ActualQPS'] 
    seconds=df.at[0, 'ActualDuration']
    title="Istio {}, {} QPS over {} seconds".format(metric, qps, seconds)

    # 4. prep file-write 
    fn = "".join(title.split())
    output_file(fn + ".html") 


    # 5. create chart 
    p = build_chart(title, metric, c, series) 


# returns pandas DF 
def read_csv(all_files):
    li = []
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        li.append(df)
    frame = pd.concat(li, axis=0, ignore_index=True)
    frame["Labels"] = [ x.split('_', 6)[-1] for x in frame['Labels']]
    return frame 

def get_series(df, metric): 
    display(df)
    modes = {'^serveronly': 'serveronly', "nomix_serveronly": "nomix_serveronly", "nomix_both": "nomix_both", "base": "base", "^both": "both"}
    series = {}
    for m, k in modes.items():
        print(k)
        rows = df[df.Labels.str.contains(m)]
        display(rows)
        vals = list(rows[metric]) 
        vals = [v/1000 for v in vals]
        vals.reverse()
        series[k] = vals 

    # get x axis (connections) 
    c = list(rows.NumThreads)
    c.sort()
    print(c)
    print(series)
    return c, series 

def build_chart(title, metric, c, series): 
    TOOLTIPS = [(metric, '$data_y')]
    p = figure(
        tools="pan,box_zoom,reset,save",
        title=title,
        tooltips=TOOLTIPS,
        plot_width=1000, plot_height=600,
        x_axis_label='connections', y_axis_label='P90 latency, millis'
    )

    # format axes 
    p.title.text_font_size = '22pt'
    p.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks
    p.yaxis.minor_tick_line_color = None  # turn off y-axis minor ticks
    p.xaxis.axis_label_text_font_size = "15pt"
    p.yaxis.axis_label_text_font_size = "15pt"
    p.xaxis.major_label_text_font_size = "13pt"
    p.yaxis.major_label_text_font_size = "13pt"
    p.xaxis.ticker = c

    # plot a different color for each series 
    colors = itertools.cycle(palette) 
    for mode, val in series.items(): 
        col = next(colors)
        p.line(c, val, line_color=col)
        p.circle(c, val, legend=mode, size=10, fill_color=col)

    p.legend.location = "top_left"
    show(p)


In [47]:
generate_chart("v12perf.csv", "p90")

,StartTime,ActualDuration,Labels,NumThreads,ActualQPS,p50,p90,p99,cpu_mili_avg_telemetry_mixer,cpu_mili_max_telemetry_mixer,mem_MB_max_telemetry_mixer,cpu_mili_avg_fortioserver_deployment_proxy,cpu_mili_max_fortioserver_deployment_proxy,mem_MB_max_fortioserver_deployment_proxy,cpu_mili_avg_ingressgateway_proxy,cpu_mili_max_ingressgateway_proxy,mem_MB_max_ingressgateway_proxy
0,2019-06-06T01:33:07.990208325Z,240,serveronly,64,1000,9315,19171,23904,1085,1462,92,725,869,128,91,105,399
1,2019-06-06T01:29:07.455748174Z,240,serveronly,32,1000,4751,11563,17148,1042,1605,91,755,903,128,95,104,398
2,2019-06-06T01:25:06.925256657Z,240,serveronly,16,1000,2880,4387,14444,1057,1154,91,899,1058,128,93,116,398
3,2019-06-06T01:21:06.463948279Z,240,serveronly,8,1000,2188,3132,12284,1138,1259,89,1012,1225,127,100,113,398
4,2019-06-06T01:17:05.961749833Z,240,serveronly,4,1000,1535,2214,9696,1035,1191,88,1014,1324,127,97,111,398
5,2019-06-06T01:13:05.516083248Z,240,serveronly,2,1000,1179,1701,9371,1086,1312,88,991,1312,127,91,112,398
6,2019-06-06T01:09:05.101911885Z,240,serveronly,1,1000,791,1141,5943,1072,1554,88,998,1205,127,95,104,398
7,2019-06-05T23:33:08.498970081Z,240,nomix_serveronly,64,1000,4594,6990,10114,10,14,66,306,362,116,99,129,386
8,2019-06-05T23:29:07.974196892Z,240,nomix_serveronly,32,1000,2735,4140,7341,9,14,66,355,404,116,91,99,386
9,2019-06-05T23:25:07.44685288Z,240,nomix_serveronly,16,1000,2099,2956,5783,10,14,66,459,543,116,87,99,386


serveronly


,StartTime,ActualDuration,Labels,NumThreads,ActualQPS,p50,p90,p99,cpu_mili_avg_telemetry_mixer,cpu_mili_max_telemetry_mixer,mem_MB_max_telemetry_mixer,cpu_mili_avg_fortioserver_deployment_proxy,cpu_mili_max_fortioserver_deployment_proxy,mem_MB_max_fortioserver_deployment_proxy,cpu_mili_avg_ingressgateway_proxy,cpu_mili_max_ingressgateway_proxy,mem_MB_max_ingressgateway_proxy
0,2019-06-06T01:33:07.990208325Z,240,serveronly,64,1000,9315,19171,23904,1085,1462,92,725,869,128,91,105,399
1,2019-06-06T01:29:07.455748174Z,240,serveronly,32,1000,4751,11563,17148,1042,1605,91,755,903,128,95,104,398
2,2019-06-06T01:25:06.925256657Z,240,serveronly,16,1000,2880,4387,14444,1057,1154,91,899,1058,128,93,116,398
3,2019-06-06T01:21:06.463948279Z,240,serveronly,8,1000,2188,3132,12284,1138,1259,89,1012,1225,127,100,113,398
4,2019-06-06T01:17:05.961749833Z,240,serveronly,4,1000,1535,2214,9696,1035,1191,88,1014,1324,127,97,111,398
5,2019-06-06T01:13:05.516083248Z,240,serveronly,2,1000,1179,1701,9371,1086,1312,88,991,1312,127,91,112,398
6,2019-06-06T01:09:05.101911885Z,240,serveronly,1,1000,791,1141,5943,1072,1554,88,998,1205,127,95,104,398


nomix_serveronly


,StartTime,ActualDuration,Labels,NumThreads,ActualQPS,p50,p90,p99,cpu_mili_avg_telemetry_mixer,cpu_mili_max_telemetry_mixer,mem_MB_max_telemetry_mixer,cpu_mili_avg_fortioserver_deployment_proxy,cpu_mili_max_fortioserver_deployment_proxy,mem_MB_max_fortioserver_deployment_proxy,cpu_mili_avg_ingressgateway_proxy,cpu_mili_max_ingressgateway_proxy,mem_MB_max_ingressgateway_proxy
7,2019-06-05T23:33:08.498970081Z,240,nomix_serveronly,64,1000,4594,6990,10114,10,14,66,306,362,116,99,129,386
8,2019-06-05T23:29:07.974196892Z,240,nomix_serveronly,32,1000,2735,4140,7341,9,14,66,355,404,116,91,99,386
9,2019-06-05T23:25:07.44685288Z,240,nomix_serveronly,16,1000,2099,2956,5783,10,14,66,459,543,116,87,99,386
10,2019-06-05T23:21:06.194907584Z,240,nomix_serveronly,8,1000,1739,2327,3564,9,10,66,548,678,116,90,106,385
11,2019-06-05T23:17:05.757094996Z,240,nomix_serveronly,4,1000,1236,1696,2413,11,13,66,680,762,116,88,98,385
12,2019-06-05T23:13:05.306923473Z,240,nomix_serveronly,2,1000,842,1160,1619,10,15,66,719,998,116,91,101,385
13,2019-06-05T23:09:04.909225885Z,240,nomix_serveronly,1,1000,647,830,1205,10,14,66,591,688,116,89,101,385


...

base


,StartTime,ActualDuration,Labels,NumThreads,ActualQPS,p50,p90,p99,cpu_mili_avg_telemetry_mixer,cpu_mili_max_telemetry_mixer,mem_MB_max_telemetry_mixer,cpu_mili_avg_fortioserver_deployment_proxy,cpu_mili_max_fortioserver_deployment_proxy,mem_MB_max_fortioserver_deployment_proxy,cpu_mili_avg_ingressgateway_proxy,cpu_mili_max_ingressgateway_proxy,mem_MB_max_ingressgateway_proxy
21,2019-06-05T21:44:54.933164293Z,240,base,64,1000,1642,5068,7886,9,12,64,7,9,115,94,108,373
22,2019-06-05T21:40:54.409943478Z,240,base,32,1000,1230,2558,5868,11,15,65,7,10,115,94,110,372
23,2019-06-05T21:36:53.948075561Z,240,base,16,1000,939,1493,3067,11,16,65,6,9,114,94,108,372
24,2019-06-05T21:32:53.522709231Z,240,base,8,1000,765,1236,2006,9,12,72,7,8,114,86,94,372
25,2019-06-05T21:28:53.089593916Z,240,base,4,1000,615,916,1573,9,11,74,7,9,114,86,96,372
26,2019-06-05T21:24:52.668371913Z,240,base,2,1000,470,646,1074,12,19,75,6,8,114,90,105,371
27,2019-06-05T21:20:52.191947976Z,240,base,1,1000,372,480,667,9,13,87,6,8,114,82,97,371


both


,StartTime,ActualDuration,Labels,NumThreads,ActualQPS,p50,p90,p99,cpu_mili_avg_telemetry_mixer,cpu_mili_max_telemetry_mixer,mem_MB_max_telemetry_mixer,cpu_mili_avg_fortioserver_deployment_proxy,cpu_mili_max_fortioserver_deployment_proxy,mem_MB_max_fortioserver_deployment_proxy,cpu_mili_avg_ingressgateway_proxy,cpu_mili_max_ingressgateway_proxy,mem_MB_max_ingressgateway_proxy
28,2019-06-05T21:11:35.637701887Z,240,both,64,1000,16971,29912,37977,2186,2871,87,1047,1239,113,82,94,370
29,2019-06-05T21:07:34.874216712Z,240,both,32,1000,8965,16374,24390,2302,3369,87,1108,1238,106,93,105,370
30,2019-06-05T21:03:34.297731912Z,240,both,16,1000,4641,8896,18398,2033,2759,87,1108,1367,104,91,101,369
31,2019-06-05T20:59:33.677990706Z,240,both,8,1000,2893,4109,15069,2039,2890,87,1192,1509,103,85,106,369
32,2019-06-05T20:55:33.223948521Z,240,both,4,1000,2180,3161,13207,2174,2520,86,1426,1844,102,80,93,368
33,2019-06-05T20:51:32.782320433Z,240,both,2,1000,1397,1795,10430,2100,2193,85,1406,1766,101,78,85,365
34,2019-06-05T20:47:32.356866182Z,240,both,1,607,1425,1919,10052,1255,1535,85,861,1048,98,85,113,365


[1, 2, 4, 8, 16, 32, 64]
{'serveronly': [1.141, 1.701, 2.214, 3.132, 4.387, 11.563, 19.171], 'nomix_serveronly': [0.83, 1.16, 1.696, 2.327, 2.956, 4.14, 6.99], 'nomix_both': [1.235, 1.741, 2.414, 2.919, 3.956, 6.659, 10.754], 'base': [0.48, 0.646, 0.916, 1.236, 1.493, 2.558, 5.068], 'both': [1.919, 1.795, 3.161, 4.109, 8.896, 16.374, 29.912]}
